In [104]:
from typing import *
from collections import namedtuple

class Expandable(object):
    def __init__(self):
        self.__data = dict()
        self.__field_properties = namedtuple('FieldPropertes', 'readonly writable')(list(), list())
    
    def __setattr__(self, name: str, value: Union[Any, tuple[Any, str]]):
        
        if hasattr(self, '_Expandable__field_properties'): 
            
            value, mode = self.__check_item(name, value)

            if mode == 'writable':
                
                if name in self.__field_properties.readonly:
                    self.__field_properties.readonly.remove(name)
                
                if name not in self.__field_properties.writable:
                    self.__field_properties.writable.append(name)
                
            elif mode == 'readonly':
                
                if name in self.__field_properties.writable:
                    self.__field_properties.writable.remove(name)
                
                if name not in self.__field_properties.readonly:
                    self.__field_properties.readonly.append(name)
                
            self.__data[name] = value
            
        if self.__is_allowed_name(name):
            self.__dict__[name] = value
            
    
    def __getitem__(self, key: Any) -> Any:
        return self.__data[key]

    def __setitem__(self, key: Any, item: Any) -> NoReturn:
        
        self.__setattr__(key, item)
    
    def __contains__(self, item: Any) -> bool:
        return item in self.__data
    
    def __iter__(self):
        return iter(self.__data)
    
    @staticmethod
    def __is_allowed_name(name: Any) -> bool:
        
        if not isinstance(name, str):
            name = str(name)
        
        for i, char in enumerate(name):
            
            if i == 0 and not char.isalpha() and not char == '_':
                return False
            elif not char.isalpha() and not char.isdigit() and not char == '_':
                return False
        
        return True
    
    def __check_item(self, name: Any, value: Any) -> tuple[Any, str]:
        deployed = False
        
        if isinstance(value, tuple):
            value, mode = value
            
            if mode in self.__field_properties._fields:
                deployed = True
            else:
                value = (value, mode)
                
        if not deployed:
            
            if name in self.__field_properties.readonly:
                raise AttributeError(f'Impossible to set a new value for a read-only field "{name}"')
            
            mode = 'writable'
        
        return value, mode
    
    def keys(self, mode: Optional[str] = 'all') -> list[Any]:
        
        if mode == 'all':
            return list(self.__data.keys())
        elif mode == 'writable':
            return self.__field_properties.writable
        elif mode == 'readonly':
            return self.__field_properties.readonly
    
    def values(self, mode: Optional[str] = 'all') -> list[Any]:
        
        if mode == 'all':
            return list(self.__data.values())
        elif mode == 'writable':
            return [self.__data[key] for key in self.__field_properties.writable]
        elif mode == 'readonly':
            return [self.__data[key] for key in self.__field_properties.readonly]
    
    def items(self, mode: Optional[str] = 'all') -> list[tuple[Any, Any]]:
        
        return list(zip(self.keys(mode), self.values(mode)))
    
    def is_writable(self, key: Any) -> bool:
        return key in self.__field_properties.writable
    
    def is_readonly(self, key: Any) -> bool:
        return key in self.__field_properties.readonly
    
    
def extendable_test():

    e = Expandable()

    e.field = 1
    e.field += 1
    print(e.field, e['field'])
    print('setting new fields works!')
    
    try:
        e.field = 3, 'readonly'
        print(e.field) 
        e.field += 1
    except AttributeError:
        print('readonly fields works!')
    
    try:
        e['field'] = e.field + 1
    except AttributeError:
        print('readonly fields works for setitem!')
    
    e.field = 4, 'writable'
    print(e.field)
    print('Changing mode for field works!')
    
    e['na_m12!3e'] = 100
    
    print(e['na_m12!3e'], 'na_m12!3e' in e.__dict__)
    
    e['new_field'] = 200
    
    print(e['new_field'], e.new_field)
    print('setitem works!')
    
    e['new_field'] = 300, 'readonly'
    
    print(e['new_field'])
    
    try:
        e['new_field'] += 1
    except AttributeError:
        try:
            e.new_field += 1
        except AttributeError:
            print('readonly fields works for setitem!')
    
    print(('new_field' in e) == True)
    print(('3eas32' in e) == False)
    print('contains works!')
    
    for field in e:
        print(field)
    
    print('iter works!')
    
    e[(0, 1)] = 20, 'readonly'
    
    print(e.keys())
    print(e.keys('writable'))
    print(e.keys('readonly'))
    print('keys works!')
    
    print(e.values())
    print(e.values('writable'))
    print(e.values('readonly'))
    print('values works!')
    
    print(e.items())
    print(e.items('writable'))
    print(e.items('readonly'))
    print('items works!')
    
    
    print(e.is_readonly('new_field'), e.is_readonly('field'))
    print(e.is_writable('new_field'), e.is_writable('field'))
    print('readonly/writable checking works!')

extendable_test()



2 2
setting new fields works!
3
readonly fields works!
readonly fields works for setitem!
4
Changing mode for field works!
100 False
200 200
setitem works!
300
readonly fields works for setitem!
True
True
contains works!
field
na_m12!3e
new_field
iter works!
['field', 'na_m12!3e', 'new_field', (0, 1)]
['field', 'na_m12!3e']
['new_field', (0, 1)]
keys works!
[4, 100, 300, 20]
[4, 100]
[300, 20]
values works!
[('field', 4), ('na_m12!3e', 100), ('new_field', 300), ((0, 1), 20)]
[('field', 4), ('na_m12!3e', 100)]
[('new_field', 300), ((0, 1), 20)]
items works!
True False
False True
readonly/writable checking works!


In [1]:
import numpy as np

a = [i for i in range(10)]

In [ ]:
a = [i for i in range(10)]
b = [10**i for i in range(5)]

for i, j in zip(a, b):
    print(i, j)

b += [0 for i in range(len(a) - len(b))]
print(b)

In [21]:
a = [i for i in range(10)]
b = [[j*10**i for j in range(10)] for i in range(10)]

for n in iter(b):
    print(n)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
[0, 100, 200, 300, 400, 500, 600, 700, 800, 900]
[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]
[0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000]
[0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000]
[0, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000, 8000000, 9000000]
[0, 10000000, 20000000, 30000000, 40000000, 50000000, 60000000, 70000000, 80000000, 90000000]
[0, 100000000, 200000000, 300000000, 400000000, 500000000, 600000000, 700000000, 800000000, 900000000]
[0, 1000000000, 2000000000, 3000000000, 4000000000, 5000000000, 6000000000, 7000000000, 8000000000, 9000000000]


In [23]:
c = np.arange(10)
d = dict(a=1, b=2, c=3)

print()

In [21]:
from typing import Iterable, Optional, Any, Callable, Union
import numpy as np


class Linked(object):
    def __init__(
            self,
            name: Optional[str] = 'Linked',
            parent: Optional[Any] = None,
            meta: Optional[dict] = None,
            options: Optional[Iterable] = None,
            child_options_generator: Optional[Union[Callable, list[Callable]]] = None,
            children: Optional[Union[Any, list[Any]]] = None
    ):
        self.name = name
        self.__siblings = None
        self.parent = parent
        if not isinstance(children, list) and issubclass(type(children), Linked):
            self.children = [children]
        elif isinstance(children, list) or children is None:
            self.children = children
        else:
            raise AttributeError(f'Children must be either subclass of Linked or list of it {issubclass(type(children), Linked)}')

        if self.children is not None:
            if issubclass(type(self.children), Linked):
                self.children._parent = self
            elif isinstance(self.children, list):
                for child in self.children:
                    child._parent = self

        if options is None or isinstance(options, Iterable):
            self._options = options
        else:
            raise AttributeError('Options must be an iterable object')
        self.child_options_generator = child_options_generator
        self.__selected_option = None
        self.meta = dict() if not isinstance(meta, dict) else meta

    def __call__(self, options: Optional[list[Any]] = None, option_index_to_choose: Optional[int] = 0):

        if options is not None and options != [] and options != ():
            self._options = options
            self.__selected_option = self.options[option_index_to_choose]

        if self.children is not None and self.child_options_generator is not None:

            if issubclass(type(self.children), Linked) and isinstance(self.child_options_generator, Callable):
                self.children(self.child_options_generator(self))

            elif isinstance(self.children, list) and isinstance(self.child_options_generator, Callable):

                for child in self.children:
                    child(self.child_options_generator(self))

            elif isinstance(self.child_options_generator, list):

                if isinstance(self.children, list):

                    if len(self.children) > len(self.child_options_generator):
                        gens = self.child_options_generator + \
                               [
                                   self.child_options_generator[-1]
                                   for _ in range(len(self.children) - len(self.child_options_generator))
                               ]
                        children = self.children

                    else:
                        gens = self.child_options_generator
                        children = self.children

                elif isinstance(self.children, Linked):
                    gens = self.child_options_generator
                    children = [self.children]

                else:
                    raise ValueError(f'Children can not belong to this type: {type(self.children)}')

                for child, gen in zip(children, gens):
                    child(gen(self))

    def __iter__(self):
        return iter([self] + self.inverse_dependencies())

    def __str__(self):
        return f'{self.name}'

    def __getitem__(self, i):
        chain = [self] + list(self.inverse_dependencies())
        return chain[i]

    @property
    def parent(self):
        return self._parent

    @parent.setter
    def parent(self, parent):
        if parent is not None and not issubclass(type(parent), Linked):
            raise AttributeError(f'Parent of Linked must be Linked or subclass of it, but it is: {type(parent)}')
        self._parent = parent
        if self._parent is not None:
            self._parent.add_children(self)

    @property
    def children(self):
        return self._children

    @children.setter
    def children(self, children):
        valid_children = issubclass(type(children), Linked)

        if isinstance(children, list):
            valid_children = True
            for child in children:

                if not issubclass(type(child), Linked):
                    valid_children = False
                    break

        if children is not None and not valid_children:
            raise AttributeError(f'Children of Linked must be list of Linked or Linked or subclass of it, '
                                 f'but it is: {type(children)}')

        self._children = children
        self.__introduce_children()

    @property
    def siblings(self):
        return self.__siblings

    @siblings.setter
    def siblings(self, value):
        raise AttributeError(f'Siblings of Linked cannot be set')

    @property
    def selected_option(self):
        return self.__selected_option

    @selected_option.setter
    def selected_option(self, value):
        raise AttributeError('Options can be selected only via \'select\' method')

    @property
    def options(self):
        return self._options

    @options.setter
    def options(self, options: Iterable):
        if self.parent is None:
            self._options = options
        else:
            raise AttributeError(
                f'Can not set options to Linked with dependencies. This Linked depends on: {self.dependencies()}')

    def __introduce_children(self):
        if isinstance(self.children, list):
            for child in self.children:
                child.__siblings = [sib for sib in self.children if sib != child]

    def add_children(self, children):
        if not isinstance(self.children, list):
            self.children = [self.children] if self.children is not None else []
        if issubclass(type(children), Linked):
            self.children.append(children)
        elif isinstance(children, list):
            for child in children:
                if not issubclass(child, Linked):
                    raise AttributeError(f'All the children must be Linked, but {type(child)} found '
                                         f'in the list of given children')
            self.children += children
        else:
            raise AttributeError(f'All the children must be Linked, but {type(children)} is given')
        self.__introduce_children()

    def remove(self):
        self.parent.children = [child for child in self.parent.children if child != self]
        for dep in self.inverse_dependencies():
            del dep
        del self

    def select(self, option, index: Optional[bool] = False, child_option_index_to_choose: Optional[int] = 0):
        if index:
            self.__selected_option = self.options[option]
        else:
            self.__selected_option = option
        self(option_index_to_choose=child_option_index_to_choose)

    def inverse_dependencies(self):
        if self.children is None:
            return []
        elif isinstance(self.children, list) and self.children:
            invdep = [child for child in self.children]
            for child in invdep:
                # Add only children which are not added already
                invdep += [
                    dep for dep in
                    child.inverse_dependencies()
                    if dep not in invdep
                ]
            return invdep
        elif issubclass(type(self.children), Linked):
            return [self.children]

    def dependencies(self):
        deps = list()
        dep = self.parent
        while dep is not None:
            deps.append(dep)
            dep = dep.parent
        return tuple(deps)


chain = Linked(
    'Layer1',
    options=np.arange(10),
    child_options_generator=[lambda l: l.options+1, lambda l: l.options**2],
    children=[
        Linked(
            name='Layer11',
            child_options_generator=lambda l: l.options*2,
            children=[
                Linked(name='Layer111'),
                Linked(
                    name='Layer112',
                    child_options_generator=lambda l: l.options*2,
                    children=Linked(name='Layer1121')
                ),
                Linked(name='Layer113'),
                Linked(name='Layer114'),
            ]
        ),
        Linked(
            name='Layer12',
            child_options_generator=lambda l: l.options+10,
            children=Linked(
                name='Layer121',
                child_options_generator=[
                    lambda l: [str(option) + '_str1_' for option in l.options],
                    lambda l: [str(option) + '_str2_' for option in l.options],
                    lambda l: [str(option) + '_str3_' for option in l.options]
                ],
                children=[
                    Linked(name='Layer1211'),
                    Linked(name='Layer1212'),
                    Linked(name='Layer1213')
                ]
            )
        ),
        Linked(
            name='Layer13',
            child_options_generator=[
                lambda l: [str(option) + 'STRRR_1__' for option in l.options],
                lambda l: [str(option) + 'STRRR_2__' for option in l.options],
                lambda l: [str(option) + 'STRRR_3__' for option in l.options]
                                     ],
            children=[
                    Linked(name='Layer1311'),
                    Linked(name='Layer1312'),
                    Linked(name='Layer1313'),
                    Linked(name='Layer1314')
                ]
        )
    ]
)

CONTROL_CHILDREN_STR = 'Layer1Layer11Layer12Layer13Layer11Layer111Layer112Layer113Layer114Layer12Layer121Layer1211Layer1212Layer1213Layer13Layer1311Layer1312Layer1313Layer1314Layer111NoneLayer112Layer1121Layer113NoneLayer114NoneLayer1121NoneLayer121Layer1211Layer1212Layer1213Layer1311NoneLayer1312NoneLayer1313NoneLayer1314NoneLayer1121NoneLayer1211NoneLayer1212NoneLayer1213None'
CONTROL_SIBLINGS_STR = 'Layer1NoneLayer11Layer12Layer13Layer12Layer11Layer13Layer13Layer11Layer12Layer111Layer112Layer113Layer114Layer112Layer111Layer113Layer114Layer113Layer111Layer112Layer114Layer114Layer111Layer112Layer113Layer1121NoneLayer121NoneLayer1311Layer1312Layer1313Layer1314Layer1312Layer1311Layer1313Layer1314Layer1313Layer1311Layer1312Layer1314Layer1314Layer1311Layer1312Layer1313Layer1121NoneLayer1211Layer1212Layer1213Layer1212Layer1211Layer1213Layer1213Layer1211Layer1212'

print('\n============ CHILDREN TEST ============\n')
children_str = ''
for link in chain:
    print(link)
    children_str += str(link)
    if not isinstance(link.children, Iterable):
        print('\t', link.children)
        children_str += str(link.children)
    else:
        for children in link.children:
            print('\t', children)
            children_str += str(children)


if children_str == CONTROL_CHILDREN_STR:
    print('\nTEST PASSED! =================\n')
else:
    print('\nTEST FAILED! =================\n')

print('\n============ SIBLINGS TEST ============\n')
siblings_str = ''
for link in chain:
    print(link)
    siblings_str += str(link)
    if not isinstance(link.siblings, Iterable):
        print('\t', link.siblings)
        siblings_str += str(link.siblings)
    else:
        for sibling in link.siblings:
            siblings_str += str(sibling)
            print('\t', sibling)

if siblings_str == CONTROL_SIBLINGS_STR:
    print('\nTEST PASSED! =================\n')
else:
    print('\nTEST FAILED! =================\n')

#
# print('\n============ ADDING TEST ============\n')
#
# print(chain.add_children(
#     Linked(
#             name='New1',
#             children=Linked(
#                 name='New11',
#                 children=[
#                     Linked(name='New111'),
#                     Linked(name='New112'),
#                     Linked(name='New113')
#                 ]
#             )
#         )
# ))
#
# print('\n============ CHILDREN TEST ============\n')
# for link in chain:
#     print(link)
#     if not isinstance(link.children, Iterable):
#         print('\t', link.children)
#     else:
#         for children in link.children:
#             print('\t', children)
# print('\n============ SIBLINGS TEST ============\n')
# for link in chain:
#     print(link)
#     if not isinstance(link.siblings, Iterable):
#         print('\t', link.siblings)
#     else:
#         for sibling in link.siblings:
#             print('\t', sibling)


# print('\n============ REMOVING TEST ============\n')
#
# print('removing ', chain[2])
# chain[2].remove()
#
# print('\n============ CHILDREN TEST ============\n')
# for link in chain:
#     print(link)
#     if not isinstance(link.children, Iterable):
#         print('\t', link.children)
#     else:
#         for children in link.children:
#             print('\t', children)
# print('\n============ SIBLINGS TEST ============\n')
# for link in chain:
#     print(link)
#     if not isinstance(link.siblings, Iterable):
#         print('\t', link.siblings)
#     else:
#         for sibling in link.siblings:
#             print('\t', sibling)

print('\n============ CALLING TEST ============\n')

chain()

for link in chain:
    print(link, link.options)


============ CHILDREN TEST ============

Layer1
	 Layer11
	 Layer12
	 Layer13
Layer11
	 Layer111
	 Layer112
	 Layer113
	 Layer114
Layer12
	 Layer121
Layer13
	 Layer1311
	 Layer1312
	 Layer1313
	 Layer1314
Layer111
	 None
Layer112
	 Layer1121
Layer113
	 None
Layer114
	 None
Layer1121
	 None
Layer121
	 Layer1211
	 Layer1212
	 Layer1213
Layer1211
	 None
Layer1212
	 None
Layer1213
	 None
Layer1311
	 None
Layer1312
	 None
Layer1313
	 None
Layer1314
	 None

TEST FAILED! =================


============ SIBLINGS TEST ============

Layer1
	 None
Layer11
	 Layer12
	 Layer13
Layer12
	 Layer11
	 Layer13
Layer13
	 Layer11
	 Layer12
Layer111
	 Layer112
	 Layer113
	 Layer114
Layer112
	 Layer111
	 Layer113
	 Layer114
Layer113
	 Layer111
	 Layer112
	 Layer114
Layer114
	 Layer111
	 Layer112
	 Layer113
Layer1121
Layer121
Layer1211
	 Layer1212
	 Layer1213
Layer1212
	 Layer1211
	 Layer1213
Layer1213
	 Layer1211
	 Layer1212
Layer1311
	 Layer1312
	 Layer1313
	 Layer1314
Layer1312
	 Layer1311
	 Layer1313
	 

/tmp/ipykernel_225946/1792571487.py:42: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if options is not None and options != [] and options != ():


In [161]:
from unittest.mock import call
import mne
import itertools

ok
